In [1]:
#https://pythonawesome.com/overview-of-the-peaks-dectection-algorithms-available-in-python/
# https://github.com/MonsieurV/py-findpeaks/blob/master/README.md
import numpy as np
import scipy.signal 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
from libs import detect_peaks
from morning.pipeline.stream.cybos.stock.db.min_excel_tick import MinExcelTick
from morning.pipeline.converter.cybos.stock.day_tick import StockDayTickConverter
from morning.pipeline.stream.cybos.stock.db.min_tick import MinTick
import pandas as pd
from morning.pipeline.stream.cybos.stock.db.min_tick import MinTick
from morning.pipeline.chooser.cybos.db.kosdaq_all_chooser import KosdaqAllChooser
from morning.back_data.fetch_stock_data import get_day_minute_period_data
from morning.pipeline.chooser.cybos.db.kosdaq_bull_chooser import KosdaqBullChooser
from datetime import date

In [2]:

#kac = KosdaqAllChooser()
#kac.set_date(date(2019, 1, 2))
from datetime import timedelta
kbc = KosdaqBullChooser(datetime.now() - timedelta(days=1), datetime.now())
codes = kbc.codes[-1].values()
codes = list(codes)[2:]

In [3]:
from_date = date(2019, 12, 4)
code_index = 0

In [4]:
def get_reversed(s):
    distance_from_mean = s.mean() - s
    return distance_from_mean + s.mean()

In [5]:
# find_peaks_cwt
def get_indexes(methods):
    price = np.array(df['avg'])
    reverse_price = np.array(get_reversed(price))
    if methods == 'peakutil':
        import peakutils.peak
        indexes = peakutils.peak.indexes(price, thres=0.5, min_dist=30)
        r_indexes = peakutils.peak.indexes( reverse_price, thres=7.0/price.max(), min_dist=10)
        return indexes, r_indexes
    elif methods == 'find_peaks_cwt':
        indexes = scipy.signal.find_peaks_cwt(price, np.arange(1, 4), max_distances=np.arange(1, 4)*2)
        r_indexes = scipy.signal.find_peaks_cwt( reverse_price, np.arange(1, 4), max_distances=np.arange(1, 4)*2)
        return indexes, r_indexes
    elif methods == 'find_peaks':
        indexes, _ = scipy.signal.find_peaks(price, height=7, distance=2.1)
        r_indexes, _ = scipy.signal.find_peaks(reverse_price, height=7, distance=2.1)
        return indexes, r_indexes
    elif methods == 'detect_peaks':
        indexes = detect_peaks.detect_peaks(price, mph=7, mpd=2)
        r_indexes = detect_peaks.detect_peaks(reverse_price, mph=7, mpd=2)
        return indexes, r_indexes



In [6]:
def show_graph(max_index, min_index):
    min_index_dates = df['date'].iloc[min_index]
    min_index_prices = df['avg'].iloc[min_index] + (df['avg'].mean() * 0.01)
    max_index_dates = df['date'].iloc[max_index]
    max_index_prices = df['avg'].iloc[max_index] + (df['avg'].mean() * 0.01)

    fig = make_subplots(rows=2, cols=1)

    fig.add_trace(go.Scatter(x=df['date'], y=df['close_price'], mode='lines', line=dict(color='black', width=1), opacity=0.9))
    fig.add_trace(go.Scatter(x=df['date'], y=df['avg'], mode='lines', opacity=1.0))
    fig.add_trace(go.Scatter(mode='markers', x=min_index_dates, y=min_index_prices, marker=dict(color='blue', size=7), opacity=0.7))
    fig.add_trace(go.Scatter(mode='markers', x=max_index_dates, y=max_index_prices, marker=dict(color='magenta', size=7), opacity=0.5))
    fig.add_trace(go.Scatter(x=df['date'], y=df['volume'], name='culmulate buy', line=dict(color='red')), row=2, col=1)
    return fig

#fig.update_layout(yaxis_tickformat='d', shapes=shapes)

In [10]:


class Receiver:
    def __init__(self):
        self.data = []
    
    def received(self, datas):
        for d in datas:
            self.data.append(d)

sdtc = StockDayTickConverter()
print(codes[code_index])
len = get_day_minute_period_data(codes[code_index], from_date, from_date)
#met = MinExcelTick('sample_data/20191129_A028300.xlsx')
met = MinTick(from_date)
met.set_target('cybos:' + codes[code_index])
met.set_output(sdtc)
code_index += 1

rcv = Receiver()
sdtc.set_output(rcv)

while met.received(None) > 0:
    pass

data = rcv.data
df = pd.DataFrame(data)
df['avg'] = df['close_price'].rolling(10).mean()
not_non = df['avg'].notna().idxmax()
if not_non > 0:
    #print(df['avg'].iloc[not_non])
    df['avg'] = df['avg'].fillna(df['avg'].iloc[not_non])

A196170
Length 381


In [11]:
show_graph(*get_indexes('peakutil'))

